# 09. PyTorch Model Deployment

What is model deployment?

Machine learning model deployment is the act of making your machine learning model(s) available to someone or something else


## Resources

Link: https://www.learnpytorch.io/09_pytorch_model_deployment/

## 0. Get setup

In [5]:
import torch
import torchvision

from torch import nn
from torchinfo import summary

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from src import get_data, helper_functions, train, utils, engine, predictions

ModuleNotFoundError: No module named 'setup_data'

In [3]:
device = "cuda" if torch.cuda.is_available() else (
    "mps" if torch.mps.is_available() else "cpu"
)

device

'mps'

## 1. Geting data

The dataset we're going to use for deploying a FoodVision Mini model is...

Pizza, steak, sushi 20% dataset (pizza steak, sushi, classes from Food101, random 20% samples)

In [ ]:
get_data.get_data()